In [3]:
!pip install -q crewai langchain langchain_core langchain_community langchain-openai wikipedia tavily-python

In [9]:
from pydantic import BaseModel
from typing import List, Dict, Any

class ResearchRequest(BaseModel):
    date_range_start: str  # Format: 'YYYY-MM-DD'
    date_range_end: str    # Format: 'YYYY-MM-DD'
    focus_area: str        # Specific area of interest
    data_sources: List[str]

class FlightDataInsight(BaseModel):
    trend_name: str
    description: str
    data_points: List[Dict[str, Any]]  # Details of data points supporting the insight
    analysis_method: str
    implications: str
    recommendations: List[str]

class FlightInnovation(BaseModel):
    innovation_name: str
    description: str
    impact: str
    supporting_data: List[Dict[str, Any]]
    references: List[str]

class FlightResearchReport(BaseModel):
    insights: List[FlightDataInsight]
    innovations: List[FlightInnovation]
    overall_analysis: str
    future_trends: List[str]
    recommendations: List[str]
    data_sources: List[str]
    additional_notes: str
    references: List[str]
    contact_information: str
    legal_disclaimer: str


In [5]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [10]:
from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools import TavilySearchResults

class CustomAgents:
    def __init__(self):
        self.OpenAIGPT4Mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.OpenAIGPT4 = ChatOpenAI(model="gpt-4o", temperature=0)
        self.tools = [
            Tool(
                name="Wikipedia",
                func=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()).run,
                description="Useful for researching historical and current events in aviation."
            ),
            TavilySearchResults(
                max_results=5,
                include_answer=True,
                include_raw_content=True
            )
        ]

    def flight_data_analyst_agent(self):
        return Agent(
            role="Flight Data Analyst",
            backstory=dedent("""You are an expert in analyzing large sets of flight data to identify trends, patterns, and anomalies."""),
            goal=dedent("""Analyze flight data within the specified date range and focus area to discover significant trends and insights that can impact the aviation industry."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def trend_analyst_agent(self):
        return Agent(
            role="Trend Analyst",
            backstory=dedent("""You specialize in interpreting data trends and predicting future developments in flight operations."""),
            goal=dedent("""Interpret analyzed data within the specified focus area to predict future trends in flight operations and suggest possible innovations."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

    def innovation_specialist_agent(self):
        return Agent(
            role="Innovation Specialist",
            backstory=dedent("""You identify and evaluate new innovations in flight data and technology that could revolutionize the industry."""),
            goal=dedent("""Identify key innovations in flight data and technology related to the focus area, and assess their potential impact on the industry."""),
            tools=self.tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def research_report_generator_agent(self):
        return Agent(
            role="Research Report Generator",
            backstory=dedent("""You compile comprehensive research reports based on data analyses and insights, adhering to detailed schemas."""),
            goal=dedent("""Using the collected data and analyses, provide a comprehensive research report matching the FlightResearchReport schema, ensuring all fields are populated with detailed and accurate information."""),
            tools=[],
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )


In [11]:
from crewai import Task
from textwrap import dedent

class CustomTasks:
    def __init__(self):
        pass

    def flight_data_analysis_task(self, agent, research_request: ResearchRequest):
        return Task(
            description=dedent(f"""
                Analyze flight operation data from **{research_request.date_range_start}** to **{research_request.date_range_end}** focusing on **{research_request.focus_area}**.
                Use the specified data sources: {', '.join(research_request.data_sources)}.
                Identify significant trends, patterns, and anomalies.
                Document each insight with detailed descriptions, supporting data points, and analysis methods used.
                **Use the provided tools to gather accurate and up-to-date information.**
            """),
            agent=agent,
            expected_output="A list of FlightDataInsight objects with detailed information as per the FlightDataInsight schema.",
        )

    def trend_analysis_task(self, agent, research_request: ResearchRequest):
        return Task(
            description=dedent(f"""
                Interpret the analyzed flight data related to **{research_request.focus_area}** to predict future trends in the aviation industry.
                Provide detailed explanations for each predicted trend, including possible implications and recommendations.
                **Use the provided tools to enhance your analysis.**
            """),
            agent=agent,
            expected_output="A set of future trends with detailed explanations and justifications.",
        )

    def innovation_identification_task(self, agent, research_request: ResearchRequest):
        return Task(
            description=dedent(f"""
                Identify recent innovations in flight data and technology within the area of **{research_request.focus_area}** that have the potential to revolutionize the aviation industry.
                Assess the impact of each innovation, backing your findings with data and references.
                **Use the provided tools to support your analysis.**
            """),
            agent=agent,
            expected_output="A list of FlightInnovation objects with detailed information as per the FlightInnovation schema.",
        )

    def research_report_task(self, agent, research_request: ResearchRequest):
        flight_research_report_schema = FlightResearchReport.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Compile a comprehensive research report based on the analyzed data and identified innovations for **{research_request.focus_area}** from **{research_request.date_range_start}** to **{research_request.date_range_end}**.
                Include the following in your report:
                - **Insights**: A detailed list of discovered insights, following the FlightDataInsight schema.
                - **Innovations**: A detailed list of identified innovations, following the FlightInnovation schema.
                - **Overall Analysis**: An in-depth analysis of the current and future state of flight operations related to **{research_request.focus_area}**.
                - **Future Trends**: Predictions about future developments in the aviation industry concerning **{research_request.focus_area}**.
                - **Recommendations**: Actionable recommendations based on your findings.
                - **Data Sources**: Include the data sources used: {', '.join(research_request.data_sources)}.
                - **Additional Notes**, **References**, **Contact Information**, **Legal Disclaimer**: Provide any additional relevant information.

                Provide your output in **JSON format** matching the **FlightResearchReport** schema below.

                **FlightResearchReport Schema**:

                {flight_research_report_schema}
            """),
            agent=agent,
            expected_output=f"The research report in JSON format matching the FlightResearchReport schema: {flight_research_report_schema}",
        )


In [12]:
from crewai import Crew
from textwrap import dedent

class FlyOpsResearcherCrew:
    def __init__(self, date_range_start, date_range_end, focus_area, data_sources):
        self.research_request = ResearchRequest(
            date_range_start=date_range_start,
            date_range_end=date_range_end,
            focus_area=focus_area,
            data_sources=data_sources
        )
        self.agents = CustomAgents()
        self.tasks = CustomTasks()

    def run(self):
        # Define agents
        flight_data_analyst_agent = self.agents.flight_data_analyst_agent()
        trend_analyst_agent = self.agents.trend_analyst_agent()
        innovation_specialist_agent = self.agents.innovation_specialist_agent()
        research_report_generator_agent = self.agents.research_report_generator_agent()

        # Define tasks with the research request
        flight_data_analysis_task = self.tasks.flight_data_analysis_task(flight_data_analyst_agent, self.research_request)
        trend_analysis_task = self.tasks.trend_analysis_task(trend_analyst_agent, self.research_request)
        innovation_identification_task = self.tasks.innovation_identification_task(innovation_specialist_agent, self.research_request)
        research_report_task = self.tasks.research_report_task(research_report_generator_agent, self.research_request)

        # Create the crew
        crew = Crew(
            agents=[
                flight_data_analyst_agent,
                trend_analyst_agent,
                innovation_specialist_agent,
                research_report_generator_agent,
            ],
            tasks=[
                flight_data_analysis_task,
                trend_analysis_task,
                innovation_identification_task,
                research_report_task,
            ],
            verbose=True,
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to the FlyOps Researcher")
    print("-------------------------------------------------------")
    print("Discovering insights and innovations in flight data. 🔍")
    date_range_start = input("Enter the start date for analysis (YYYY-MM-DD): ")
    date_range_end = input("Enter the end date for analysis (YYYY-MM-DD): ")
    focus_area = input("Enter the focus area for analysis (e.g., delays, fuel efficiency): ")
    data_sources_input = input("Enter the data sources to use (separated by commas): ")
    data_sources = [source.strip() for source in data_sources_input.split(',')]

    researcher = FlyOpsResearcherCrew(
        date_range_start=date_range_start.strip(),
        date_range_end=date_range_end.strip(),
        focus_area=focus_area.strip(),
        data_sources=data_sources
    )
    results = researcher.run()
    print("\n\n########################")
    print("## Here is your research report:")
    print("########################\n")
    print(results)


## Welcome to the FlyOps Researcher
-------------------------------------------------------
Discovering insights and innovations in flight data. 🔍
Enter the start date for analysis (YYYY-MM-DD): 2024-10-10
Enter the end date for analysis (YYYY-MM-DD): 2024-10-12
Enter the focus area for analysis (e.g., delays, fuel efficiency): fuel efficiency
Enter the data sources to use (separated by commas): public datasets
# Agent: Flight Data Analyst
## Task: 
Analyze flight operation data from **2024-10-10** to **2024-10-12** focusing on **fuel efficiency**.
Use the specified data sources: public datasets.
Identify significant trends, patterns, and anomalies.
Document each insight with detailed descriptions, supporting data points, and analysis methods used.
**Use the provided tools to gather accurate and up-to-date information.**



# Agent: Flight Data Analyst
## Thought: I need to gather current and relevant information regarding flight operations, specifically focusing on fuel efficiency dur

In [13]:
results

CrewOutput(raw='{\n  "insights": [\n    {\n      "trend_name": "Sustainable Aviation Fuels (SAF)",\n      "description": "Sustainable Aviation Fuels are biofuels used to power aircraft, significantly reducing CO2 emissions by up to 80% compared to traditional jet fuels. SAF can be produced from various feedstocks, including waste oils and non-food crops.",\n      "data_points": [\n        {\n          "potential_emission_reduction": "80%",\n          "current_production": "600 million liters in 2023, representing 0.2% of global jet fuel use.",\n          "challenges": "Feedstock constraints and high production costs."\n        }\n      ],\n      "analysis_method": "Comparative analysis of emission reductions and production data.",\n      "implications": "SAF offers a significant reduction in carbon emissions, but scaling production is challenging due to feedstock and cost issues.",\n      "recommendations": [\n        "Airlines should invest in SAF infrastructure and collaborate with p

In [15]:
type(results)

crewai.crews.crew_output.CrewOutput

In [16]:
print(results.__dict__)

{'raw': '{\n  "insights": [\n    {\n      "trend_name": "Sustainable Aviation Fuels (SAF)",\n      "description": "Sustainable Aviation Fuels are biofuels used to power aircraft, significantly reducing CO2 emissions by up to 80% compared to traditional jet fuels. SAF can be produced from various feedstocks, including waste oils and non-food crops.",\n      "data_points": [\n        {\n          "potential_emission_reduction": "80%",\n          "current_production": "600 million liters in 2023, representing 0.2% of global jet fuel use.",\n          "challenges": "Feedstock constraints and high production costs."\n        }\n      ],\n      "analysis_method": "Comparative analysis of emission reductions and production data.",\n      "implications": "SAF offers a significant reduction in carbon emissions, but scaling production is challenging due to feedstock and cost issues.",\n      "recommendations": [\n        "Airlines should invest in SAF infrastructure and collaborate with producer

In [18]:
results.raw

'{\n  "insights": [\n    {\n      "trend_name": "Sustainable Aviation Fuels (SAF)",\n      "description": "Sustainable Aviation Fuels are biofuels used to power aircraft, significantly reducing CO2 emissions by up to 80% compared to traditional jet fuels. SAF can be produced from various feedstocks, including waste oils and non-food crops.",\n      "data_points": [\n        {\n          "potential_emission_reduction": "80%",\n          "current_production": "600 million liters in 2023, representing 0.2% of global jet fuel use.",\n          "challenges": "Feedstock constraints and high production costs."\n        }\n      ],\n      "analysis_method": "Comparative analysis of emission reductions and production data.",\n      "implications": "SAF offers a significant reduction in carbon emissions, but scaling production is challenging due to feedstock and cost issues.",\n      "recommendations": [\n        "Airlines should invest in SAF infrastructure and collaborate with producers to sec

In [19]:
import json

parsed_data = json.loads(results.raw)

print(parsed_data)

{'insights': [{'trend_name': 'Sustainable Aviation Fuels (SAF)', 'description': 'Sustainable Aviation Fuels are biofuels used to power aircraft, significantly reducing CO2 emissions by up to 80% compared to traditional jet fuels. SAF can be produced from various feedstocks, including waste oils and non-food crops.', 'data_points': [{'potential_emission_reduction': '80%', 'current_production': '600 million liters in 2023, representing 0.2% of global jet fuel use.', 'challenges': 'Feedstock constraints and high production costs.'}], 'analysis_method': 'Comparative analysis of emission reductions and production data.', 'implications': 'SAF offers a significant reduction in carbon emissions, but scaling production is challenging due to feedstock and cost issues.', 'recommendations': ['Airlines should invest in SAF infrastructure and collaborate with producers to secure supply chains.', 'Governments should provide incentives for SAF development.']}, {'trend_name': 'Continuous Improvement in

In [20]:
type(parsed_data)

dict